In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph , START , END
from typing import TypedDict, Literal

In [2]:
model = ChatOpenAI(model='gpt-5-nano')

In [3]:
class EmailState(TypedDict):
    email_text:str
    reply:str
    category:str 

In [7]:
def classify_email(state:EmailState):
    prompt=f"Classify this email as personal, work, or spam. Reply with one word only.\n\nEmail:\n{email_text}"
    category=model.invoke(prompt).content 
    return {"category":category} 

def personal_reply(state: EmailState):
    prompt = f"Write a friendly and casual reply to this personal email:\n\n{state['email_text']}"
    reply = model.invoke(prompt).content
    return {"reply": reply}

def work_reply(state: EmailState):
    prompt = f"Write a formal and professional reply to this work email:\n\n{state['email_text']}"
    reply = model.invoke(prompt).content
    return {"reply": reply}

def spam_reply(state: EmailState):
    return {"reply": "No reply sent (spam detected)."}

#Conditional Function 
def check_category(state: EmailState) -> Literal["personal_reply", "work_reply", "spam_reply"]:
    if state["category"] == "personal":
        return "personal_reply"
    elif state["category"] == "work":
        return "work_reply"
    else:
        return "spam_reply"

In [8]:
graph = StateGraph(EmailState)

graph.add_node("classify_email", classify_email)
graph.add_node("personal_reply", personal_reply)
graph.add_node("work_reply", work_reply)
graph.add_node("spam_reply", spam_reply)

graph.add_edge(START, "classify_email")
graph.add_conditional_edges("classify_email", check_category)
graph.add_edge("personal_reply", END)
graph.add_edge("work_reply", END)
graph.add_edge("spam_reply", END)

workflow = graph.compile()

In [ ]:
email_text=""" 
Subject: Project Timeline Adjustment  
Hi Sujal,  
After reviewing the current progress on the data migration project, we’d like to adjust the delivery timeline.  
Could you please provide an updated schedule by Thursday so we can realign the client expectations?  
Best regards,  
Riya Mehta  
Project Manager, TechNova Solutions
"""
output=workflow.invoke({"email_text":email_text})

In [17]:
print(output["category"])

work


In [18]:
print(output["reply"])

Subject: Re: Project Timeline Adjustment

Dear Riya,

Thank you for the update. I understand the need to adjust the delivery timeline and will prepare an updated schedule that reflects the current data migration progress and realigned client expectations.

I will deliver the revised plan by Thursday, September 18, 2025, and it will include:
- Updated milestone dates and the revised critical path
- Dependencies and resource allocations
- Assumptions and risk mitigation notes
- Contingency options (baseline and conservative) to cover potential variances

I can provide the update in both an executive summary and a detailed, Gantt-friendly format. If there are any specific client constraints or priorities to incorporate, please share them; otherwise I will align with the latest scope and internal capacity.

Would you prefer a brief 15-minute walkthrough to review the changes, or is a written document sufficient for now?

Best regards,
Sujal [Last Name]
[Title]
TechNova Solutions
[Phone]
[E

In [13]:
email_text="""Subject: Congratulations! You Won  
Dear User,  
You have been selected as the lucky winner of $1,000,000!  
Click this link immediately to claim your prize: http://fakeprize.com  
Act now or your reward will expire today!  
Best regards,  
MegaPrize Lottery Team")"""

result = workflow.invoke({"email_text":email_text})

In [15]:
print(result["category"])

spam
